In [ ]:
import os
import sys
import fnmatch
import numpy as np
import pandas as pd
import json
import gzip
import pickle
import csv
import scipy.sparse
Xauth = None
from collections import defaultdict
import matplotlib.pyplot as plt

In [ ]:
# setup the update to work despite the broken scipy documentation
try:
    a = scipy.sparse.dok_matrix((10,10))
    a.update({(0,0):1.0})
    scipy.sparse.dok_matrix.my_update = scipy.sparse.dok_matrix.update
except:
    a = scipy.sparse.dok_matrix((10,10))
    a._update({(0,0):1.0})
    scipy.sparse.dok_matrix.my_update = scipy.sparse.dok_matrix._update

In [ ]:
with gzip.open('useful_venue_list.pkl.gz','rb') as fp:
    all_venues = pickle.load(fp)
with gzip.open('useful_authors_list.pkl.gz','rb') as fp:
    all_authors = pickle.load(fp)
with gzip.open('useful_papers.pkl.gz','rb') as fp:
    all_papers = pickle.load(fp)


In [ ]:
min_year = all_papers[0][6]
max_year = all_papers[-1][6]
span_years = max_year - min_year + 1
print(min_year,max_year,span_years)
conf_idx = {v:i for i,v in enumerate(all_venues)}
name_idx = {v:i for i,v in enumerate(all_authors)}
n_confs = len(all_venues)
n_auths = len(all_authors)
n_papers = len(all_papers)
print(n_confs,n_auths,n_papers)

In [ ]:
clf =  np.load('clf_gold.pkl.npy')
years_per_conf = clf.shape[0]//n_confs
YEAR_BLOCKS = span_years//years_per_conf
clf[2323]


In [ ]:
import scipy.sparse
import gc
if Xauth is None:
    valid_ns = set()
    for paper in all_papers:
        tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
        n = len(authors)
        valid_ns.add(n)
    
    per_author_val = {}
    for n in valid_ns:
        author_scores = 1/(np.arange(n)+1)
        per_author_val[n] = author_scores/author_scores.sum()
        
    count_vecs = {}
    paper_vecs = []
    for paper in all_papers:
        tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
        n = len(authors)
        j = years_per_conf*conf_idx[venue] + (year-min_year)//YEAR_BLOCKS

        author_scores = per_author_val[n]
        paper_vecs.append([(name_idx[a],j,v) for a,v in zip(authors,author_scores)])
        
    Xauth = scipy.sparse.dok_matrix((n_auths,years_per_conf*n_confs))
    xdict = {}
  
    for paper_vec in paper_vecs:
        for i,j,v in paper_vec:
            xdict[(i,j)] = v + xdict.get((i,j),0)

    Xauth.my_update(xdict)
            
    Xauth = scipy.sparse.csr_matrix(Xauth)
    paper_vec = []
    xdict = {}
    gc.collect()
    auth_years = np.ones((n_auths,2)) * np.array([3000,1000]) 
    for paper in all_papers:
        tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
        for a in authors:
            i = name_idx[a]
            auth_years[i,0] = min(auth_years[i,0],year)
            auth_years[i,1] = max(auth_years[i,1],year)

In [ ]:
cmu_uni = pd.read_csv('other_ranks/cmu_faculty.csv')
cmu_uni = cmu_uni.fillna('Other')
cmu_uni = cmu_uni[cmu_uni.dept == 'RI']
#uni_names = ['Andrea Tagliasacchi','Paul G. Kry']#['Xuemin Shen','H. Vincent Poor','Kang G. Shin','Mohamed-Slim Alouini','Lajos Hanzo']#list(cmu_uni.name)
#uni_names = list(faculty_affil[faculty_affil.affiliation == 'Johns Hopkins University'].name)
uni_names = list(cmu_uni.name)
print(len(uni_names))
conf_counts = {}
conf_counts_value = {}

for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    if year < 2000:
        continue
    n = len(authors)
    for a in authors:
        if a in uni_names:
            conf_counts[venue] = 1/n + conf_counts.get(venue,0)
            conf_counts_value[venue] = clf[years_per_conf*(conf_idx[venue]) + (year-min_year)//YEAR_BLOCKS]/n + conf_counts_value.get(venue,0)
conf_counts_value = {k: v/conf_counts[k] for k,v in conf_counts_value.items()}
ri_fav_confs = [(conf_counts[_[1]]*conf_counts_value[_[1]],_[1],conf_counts[_[1]],conf_counts_value[_[1]]) for _ in sorted([(v,k) for k,v in conf_counts.items() if v > 0],reverse=True)]

In [ ]:
confs_to_filter = [_[1] for _ in sorted(ri_fav_confs,reverse=True) if _[-2] >= 1.0]

In [ ]:
only_after = 1970
start_year_idx = max(0,only_after-min_year)//YEAR_BLOCKS

year_filter = np.zeros_like(clf).reshape((-1,years_per_conf))

if False: # filter confs
    for conf in confs_to_filter:
        year_filter[conf_idx[conf],start_year_idx:] = 1
else:
    year_filter[:,start_year_idx:] = 1
total_scores = Xauth.dot(clf * year_filter.reshape((-1)))


year_filter = np.zeros_like(clf).reshape((-1,years_per_conf))

for conf in confs_to_filter:
    year_filter[conf_idx[conf],start_year_idx:] = 1
ri_scores = Xauth.dot(clf * year_filter.reshape((-1)))


In [ ]:
ri_scores.shape,Xauth.shape

In [ ]:
best_idx = np.argsort(total_scores)[::-1]
for k in range(10):
    idx = best_idx[k]
    print('{:30s}\t{:.2f}'.format(all_authors[idx],total_scores[idx]))

# New APM?

In [ ]:
import scipy.sparse
import gc
Xauth = None
unique_auth = set()


for paper in all_papers:
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    auth_set = tuple(sorted(authors))
    unique_auth.add(auth_set)

unique_auth = sorted(list(unique_auth))
unique_auth_idx = {v:k for k,v in enumerate(unique_auth)}

n_unique_auths = len(unique_auth)
Xauth = scipy.sparse.dok_matrix((n_unique_auths,n_auths))
y = np.zeros(n_unique_auths)
xdict = {}


for i,paper in enumerate(all_papers):
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    auth_set = tuple(sorted(authors))
    k = unique_auth_idx[auth_set]
    n = len(authors)
    for a,v in zip(authors,per_author_val[n]):
        j = name_idx[a]
        # 1/n, 1
        xdict[(k,j)] = 1/n
    k_idx = years_per_conf*conf_idx[venue] + (year-min_year)//YEAR_BLOCKS
    y[k] += clf[k_idx]
Xauth.my_update(xdict)

Xauth = scipy.sparse.csr_matrix(Xauth)

In [ ]:
import os
DEBUG_THING = False
FI =  bool(int(os.environ.get('FIT_INTERCEPT',0))) #7

new_set = [3e-3,1e-1]
new_set = [6e-4, 1.8]
new_set = [1e-7, 326]

In [ ]:
from sklearn.linear_model import SGDRegressor
clf2 = SGDRegressor('huber',alpha=new_set[0],epsilon=new_set[1],penalty='l2',tol=1e-6,max_iter=100,average=True,verbose=1,fit_intercept=FI)
clf2.fit(Xauth,y)

In [ ]:
if DEBUG_THING:
    X = Xauth
    rand_idx = np.arange(X.shape[0])
    HOLDOUT=int(round(0.85*rand_idx.shape[0]))

    np.random.shuffle(rand_idx)


    X_train = X[rand_idx[:HOLDOUT]]
    X_test = X[rand_idx[HOLDOUT:]]
    y_train = y[rand_idx[:HOLDOUT]]
    y_test = y[rand_idx[HOLDOUT:]]

In [ ]:
if DEBUG_THING:
    def eval_reg(regW):
        regW2 = np.exp(regW)[0]
        eps = np.exp(regW)[1]
        clf2 = SGDRegressor('huber',alpha=regW2,penalty='l2',
                            tol=1e-6,max_iter=50,average=True,verbose=0,
                            fit_intercept=True,epsilon=eps)
        #clf2.fit(Xauth,yt)
        clf2.fit(X_train,y_train)

        diff = clf2.predict(X_test) - y_test
        return np.sqrt(abs(diff).mean() * (diff**2).mean())
    x0 = np.log(new_set)
    print(eval_reg(x0))

In [ ]:
import cma

from multiprocessing import Pool
x0 = np.log(new_set)
if DEBUG_THING:
    with Pool(8) as p:
        es = cma.CMAEvolutionStrategy(x0,0.3,{'popsize_factor':2.5})
        while not es.stop():
            solutions = es.ask()

            np.random.shuffle(rand_idx)

            X_train = X[rand_idx[:HOLDOUT]]
            X_test = X[rand_idx[HOLDOUT:]]
            y_train = y[rand_idx[:HOLDOUT]]
            y_test = y[rand_idx[HOLDOUT:]]

            def eval_reg(regW):
                regW2 = np.exp(regW)[0]
                eps = np.exp(regW)[1]
                clf2 = SGDRegressor('huber',alpha=regW2,penalty='l2',
                                    tol=1e-9,max_iter=50,average=True,verbose=0,
                                    fit_intercept=False,epsilon=eps)
                #clf2.fit(Xauth,yt)
                clf2.fit(X_train,y_train)

                diff = clf2.predict(X_test) - y_test
                return np.sqrt(abs(diff).mean() * (diff**2).mean())
            res = p.map(eval_reg, solutions)
            #res = [eval_reg(x) for x in solutions]
            es.tell(solutions, res)
            es.logger.add() 
            es.disp()

In [ ]:
if DEBUG_THING:
    np.exp(es.mean)

In [ ]:
napm_weights = clf2.coef_
napm_weights = (napm_weights-napm_weights.mean())/napm_weights.std()
_ = plt.hist(napm_weights,100,log=True)
print((clf2.intercept_-napm_weights.mean())/napm_weights.std())

In [ ]:
curious_names = ['Xiaolong Wang 0004','Judy Hoffman','Paris Siminelakis','Roie Levin','Leonid Keselman',
                 'Nicholas Rhinehart','Vincent Sitzmann','Siddharth Ancha','Xingyu Lin',
                 'Humphrey Hu','Aditya Dhawale','Nick Gisolfi','Andrey Kurenkov','Micah Corah',
                 'David F. Fouhey','Chelsea Finn','Akshara Rai','Ankit Bhatia','Xuning Yang',
                 'Lerrel Pinto','Alexander Spitzer','Roberto Shu','Allison Del Giorno','Nadine Chang',
                 'Justin Johnson 0001','Kumar Shaurya Shankar','Ellen A. Cappo','Hunter Goforth',
                 'Amir Roshan Zamir','Jonathan T. Barron','Dorsa Sadigh','Derek Hoiem','Vaggos Chatziafratis',
                 'Brian Okorn','David Held','Adam W. Harley','Hsiao-Yu Fish Tung','Tess Lee Hellebrekers','Mark Sheinin']

In [ ]:
uni_scores = sorted([(napm_weights[name_idx[name]],name) for name in all_authors],reverse=True)
uni_scores[:50]

In [ ]:
uni_scores = sorted([(napm_weights[name_idx[name]],name) for name in uni_names if name in name_idx],reverse=True)
uni_scores

# Build the APM matrix

In [ ]:
Xauth = None
gc.collect()

In [ ]:
Xauth = scipy.sparse.dok_matrix((n_papers,n_auths))
y = np.zeros(n_papers)
xdict = {}

for i,paper in enumerate(all_papers):
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    n = len(authors)
    for a in authors:
        j = name_idx[a]
        xdict[(i,j)] = 1/n
    ji = years_per_conf*conf_idx[venue] + (year-min_year)//YEAR_BLOCKS
    y[i] = clf[ji]

Xauth.my_update(xdict)

Xauth = scipy.sparse.csr_matrix(Xauth)
paper_vec = []
xdict = {}
gc.collect()

In [ ]:
_ = plt.hist(y,50)
plt.figure()
_ = plt.hist((y-y.mean())/y.std(),50)
y.std(),y.mean()
plt.figure()
yt = np.copy(y)
where = np.where(y > 0)
yt2 = np.log(yt[where])
yt[where] = (yt2-yt2.mean())/yt2.std()
yt = np.clip(yt,-3.5,3.5)
_ = plt.hist(yt,50)


In [ ]:
import os
FI =  bool(int(os.environ.get('FIT_INTERCEPT',0))) #7

In [ ]:
from sklearn.linear_model import SGDRegressor
clf2 = SGDRegressor('huber',alpha=1e-3,penalty='l2',tol=1e-6,max_iter=1000,average=True,verbose=1,fit_intercept=FI)
#clf2.fit(Xauth,yt)
clf2.fit(Xauth,y)

In [ ]:
apm_weights = clf2.coef_
apm_weights = (apm_weights-apm_weights.mean())/apm_weights.std()
_ = plt.hist(apm_weights,100,log=True)
print((clf2.intercept_-apm_weights.mean())/apm_weights.std())

# cumulative, pairwise apm
Some questions I have
* Should you get 1 and 0.5 as the coefficients?
* Should you get per_paper or total credit?

In [ ]:
import itertools

pairwise_authors = {}

for i,paper in enumerate(all_papers):
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    n = len(authors)
    authors = sorted(authors)
    for ap in itertools.combinations(authors, 2):
        if not ap in pairwise_authors:
            pairwise_authors[ap] = len(pairwise_authors)
n_pairwise_authors = len(pairwise_authors)

In [ ]:
Xauth = scipy.sparse.dok_matrix((n_auths+n_pairwise_authors,n_auths))
y = np.zeros(n_auths+n_pairwise_authors)
xdict = {}

for i,paper in enumerate(all_papers):
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    n = len(authors)
    ji = years_per_conf*conf_idx[venue] + (year-min_year)//YEAR_BLOCKS
    per_person_credit = clf[ji]/n
    for a in authors:
        j = name_idx[a]
        xdict[(j,j)] = 1
        y[j] += per_person_credit
    
    authors_s = sorted(authors)
    for ap in itertools.combinations(authors_s, 2):
        j2 = pairwise_authors[ap] + n_auths
        a1 = name_idx[ap[0]]
        a2 = name_idx[ap[1]]
        xdict[(j2,a1)] = 0.5
        xdict[(j2,a2)] = 0.5
        y[j2] += per_person_credit


Xauth.my_update(xdict)

Xauth = scipy.sparse.csr_matrix(Xauth)
paper_vec = []
xdict = {}
gc.collect()

In [ ]:
from sklearn.linear_model import SGDRegressor
clf3 = SGDRegressor('huber',alpha=1e-3,penalty='l2',tol=1e-6,max_iter=1000,average=True,verbose=1,fit_intercept=FI)
clf3.fit(Xauth,y)

In [ ]:
pw_apm_weights = clf3.coef_
pw_apm_weights = (pw_apm_weights-pw_apm_weights.mean())/pw_apm_weights.std()
_ = plt.hist(pw_apm_weights,100,log=True)
print((clf3.intercept_-pw_apm_weights.mean())/pw_apm_weights.std())

In [ ]:
Xauth = scipy.sparse.dok_matrix((n_auths+n_pairwise_authors,n_auths))
y = np.zeros(n_auths+n_pairwise_authors)
xdict = {}

for i,paper in enumerate(all_papers):
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    n = len(authors)
    ji = years_per_conf*conf_idx[venue] + (year-min_year)//YEAR_BLOCKS
    per_person_credit = clf[ji]/n
    for a in authors:
        j = name_idx[a]
        xdict[(j,j)] = 1 + xdict.get((j,j),0)
        y[j] += per_person_credit
    
    authors_s = sorted(authors)
    for ap in itertools.combinations(authors_s, 2):
        j2 = pairwise_authors[ap] + n_auths
        a1 = name_idx[ap[0]]
        a2 = name_idx[ap[1]]
        xdict[(j2,a1)] = 0.5 + xdict.get((j2,a1),0)
        xdict[(j2,a2)] = 0.5 + xdict.get((j2,a2),0)
        y[j2] += per_person_credit


Xauth.my_update(xdict)

Xauth = scipy.sparse.csr_matrix(Xauth)
paper_vec = []
xdict = {}
gc.collect()

In [ ]:
from sklearn.linear_model import SGDRegressor
clf4 = SGDRegressor('huber',alpha=1e-3,penalty='l2',tol=1e-6,max_iter=1000,average=True,verbose=1,fit_intercept=FI)
clf4.fit(Xauth,y)

In [ ]:
pweff_apm_weights = clf4.coef_
pweff_apm_weights = (pweff_apm_weights-pweff_apm_weights.mean())/pweff_apm_weights.std()
_ = plt.hist(pweff_apm_weights,100,log=True)
print((clf4.intercept_-pweff_apm_weights.mean())/pweff_apm_weights.std())

In [ ]:
if False:
    Xauth = scipy.sparse.dok_matrix((n_auths+n_pairwise_authors,n_auths + 1))
    y = np.zeros(n_auths+n_pairwise_authors)
    xdict = {}

    for i,paper in enumerate(all_papers):
        tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
        n = len(authors)
        ji = years_per_conf*conf_idx[venue] + (year-min_year)//YEAR_BLOCKS
        per_person_credit = clf[ji]
        for a in authors:
            j = name_idx[a]
            xdict[(j,j)] = 1 + xdict.get((j,j),0)
            # an unknown author
            xdict[(j,n_auths)] = (n-1) + xdict.get((j,n_auths),0)

            y[j] += per_person_credit


        authors_s = sorted(authors)
        for ap in itertools.combinations(authors_s, 2):
            j2 = pairwise_authors[ap] + n_auths + 1
            a1 = name_idx[ap[0]]
            a2 = name_idx[ap[1]]
            xdict[(j2,a1)] = 1.0 + xdict.get((j2,a1),0)
            xdict[(j2,a2)] = 1.0 + xdict.get((j2,a2),0)
            y[j2] += per_person_credit
            xdict[(j2,n_auths)] = (n-2) + xdict.get((j2,n_auths),0)



    Xauth.my_update(xdict)

    Xauth = scipy.sparse.csr_matrix(Xauth)
    paper_vec = []
    xdict = {}
    gc.collect()

In [ ]:
if False:
    from sklearn.linear_model import SGDRegressor
    clf5 = SGDRegressor('huber',alpha=1e-3,penalty='l2',tol=1e-6,max_iter=25,average=True,verbose=1,fit_intercept=FI)
    clf5.fit(Xauth,y)

In [ ]:
if False:
    pwunk_apm_weights = clf5.coef_[:n_auths]
    pwunk_apm_weights = (pwunk_apm_weights-pwunk_apm_weights.mean())/pwunk_apm_weights.std()
    _ = plt.hist(pwunk_apm_weights,100,log=True)
    print((clf5.coef_[n_auths]-pwunk_apm_weights.mean())/pwunk_apm_weights.std())

In [ ]:
# get author year counts
auth_pro_years = defaultdict(set)

for i,paper in enumerate(all_papers):
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    n = len(authors)
    for a in authors:
        auth_pro_years[a].add(year)
            
    authors_s = sorted(authors)
    for ap in itertools.combinations(authors_s, 2):
        auth_pro_years[ap].add(year)


In [ ]:
auth_pro_years_count = {k: len(v) for k,v in auth_pro_years.items()}
auth_pro_years = None
gc.collect()

In [ ]:
year_span2 = np.array([auth_pro_years_count[n] for n in all_authors])

In [ ]:
Xauth = scipy.sparse.dok_matrix((n_auths+n_pairwise_authors,n_auths))
y = np.zeros(n_auths+n_pairwise_authors)
xdict = {}

for i,paper in enumerate(all_papers):
    tag,title, authors, venue, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    n = len(authors)
    ji = years_per_conf*conf_idx[venue] + (year-min_year)//YEAR_BLOCKS
    credit = clf[ji]
    author_scores = per_author_val[n]

    for a,c in zip(authors,author_scores):
        j = name_idx[a]
        xdict[(j,j)] = c + xdict.get((j,j),0)

        y[j] += c*credit

    authors_s = sorted([(a,c) for a,c in zip(authors,author_scores)])
    for ap in itertools.combinations(authors_s, 2):
        a1,c1 = ap[0]
        a2,c2 = ap[1] 
        j2 = pairwise_authors[(a1,a2)] + n_auths
        a1 = name_idx[a1]
        a2 = name_idx[a2]
        xdict[(j2,a1)] = c1 + xdict.get((j2,a1),0)
        xdict[(j2,a2)] = c2 + xdict.get((j2,a2),0)
        y[j2] += (c1+c2)*credit


Xauth.my_update(xdict) 

Xauth = scipy.sparse.csr_matrix(Xauth)
paper_vec = []
xdict = {}
gc.collect()

In [ ]:
yn = np.zeros_like(y)
for k,v in name_idx.items():
    yn[v] = auth_pro_years_count[k]
for k,v in pairwise_authors.items():
    yn[n_auths+v] = auth_pro_years_count[k]

In [ ]:
from sklearn.linear_model import SGDRegressor
clf5 = SGDRegressor('huber',alpha=1e-3,penalty='l2',tol=1e-6,max_iter=25,average=True,verbose=1,fit_intercept=FI)
clf5.fit(Xauth,y/yn)

In [ ]:
pwunk_apm_weights = clf5.coef_
pwunk_apm_weights = (pwunk_apm_weights-pwunk_apm_weights.mean())/pwunk_apm_weights.std()
_ = plt.hist(pwunk_apm_weights,100,log=True)
print((clf5.intercept_-pwunk_apm_weights.mean())/pwunk_apm_weights.std())

In [ ]:
#year_span2 = (auth_years[:,1] - auth_years[:,0]) + 1
year_span2 = np.maximum(1,year_span2)

eff_scores = total_scores/year_span2
df_clfs2 = pd.DataFrame(np.vstack([total_scores,eff_scores,apm_weights,pw_apm_weights,pweff_apm_weights,pwunk_apm_weights]).T,columns=['Total','Per Year','APM','PW APM','PW Eff APM','PW APM Yearly'])
#print(df_clfs2.corr('spearman').to_latex())
corr = df_clfs2.corr('spearman')
print(corr.mean(0))
corr

In [ ]:
best_idx = np.argsort(napm_weights)[::-1]
for k in range(180):
    idx = best_idx[k]
    name = all_authors[idx]
    if name + ' 0001' in name_idx:
        print('.',end='')
        #continue
    print('{:30s}\t{:.2f}\t{:.4f}'.format(all_authors[idx],total_scores[idx],napm_weights[idx]))

In [ ]:
best_idx = np.argsort(apm_weights)[::-1]
for k in range(180):
    idx = best_idx[k]
    name = all_authors[idx]
    if name + ' 0001' in name_idx:
        print('.',end='')
        #continue
    print('{:30s}\t{:.2f}\t{:.4f}'.format(all_authors[idx],total_scores[idx],apm_weights[idx]))

In [ ]:
best_idx = np.argsort(pw_apm_weights)[::-1]
for k in range(180):
    idx = best_idx[k]
    name = all_authors[idx]
    if name + ' 0001' in name_idx:
        print('.',end='')
        #continue
    print('{:30s}\t{:.2f}\t{:.2f}\t{:.2f}'.format(name,total_scores[idx],pw_apm_weights[idx],apm_weights[idx]))

In [ ]:
best_idx = np.argsort(pweff_apm_weights)[::-1]
for k in range(180):
    idx = best_idx[k]
    name = all_authors[idx]
    if name + ' 0001' in name_idx:
        print('.',end='')
        #continue
    print('{:30s}\t{:.2f}\t{:.2f}\t{:.2f}'.format(name,total_scores[idx],pweff_apm_weights[idx],apm_weights[idx]))

In [ ]:
best_idx = np.argsort(pwunk_apm_weights)[::-1]
for k in range(180):
    idx = best_idx[k]
    name = all_authors[idx]
    if name + ' 0001' in name_idx:
        print('.',end='')
        #continue
    print('{:30s}\t{:.2f}\t{:.2f}\t{:.2f}'.format(name,total_scores[idx],pwunk_apm_weights[idx],apm_weights[idx]))

In [ ]:
uni_scores = sorted([(pwunk_apm_weights[name_idx[name]],name) for name in uni_names if name in name_idx],reverse=True)
uni_scores

In [ ]:
uni_scores = sorted([(pweff_apm_weights[name_idx[name]],name) for name in uni_names if name in name_idx],reverse=True)
uni_scores

In [ ]:
uni_scores = sorted([(pw_apm_weights[name_idx[name]],name) for name in uni_names if name in name_idx],reverse=True)
uni_scores

In [ ]:
uni_scores = sorted([(napm_weights[name_idx[name]],name) for name in uni_names if name in name_idx],reverse=True)
uni_scores

In [ ]:
uni_scores = sorted([(apm_weights[name_idx[name]],name) for name in uni_names if name in name_idx],reverse=True)
uni_scores

In [ ]:
uni_scores = sorted([(total_scores[name_idx[name]],name) for name in uni_names if name in name_idx],reverse=True)
uni_scores

In [ ]:
np.save('napm'+str(FI),napm_weights)
np.save('pwapm'+str(FI),pw_apm_weights)
np.save('pweffapm'+str(FI),pweff_apm_weights)
np.save('pwunkapm'+str(FI),pwunk_apm_weights)
np.save('apm'+str(FI),apm_weights)
np.save('total',total_scores)